# MongoDB Vector Search - Vector Quantization - New Data

This notebook is a companion to the [Vector Quantization](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-quantization/#how-to-ingest-pre-quantized-vectors) tutorial. Refer to the page for set-up instructions and detailed explanations.

This notebook takes you through how to pre-quantize and ingest your vectors for vector search from **existing data in Atlas** by using the `voyage-4-large` model from [Voyage AI](https://www.voyageai.com).

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/quantization/new-data.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
pip install --quiet --upgrade voyageai pymongo

In [ ]:
import os
import voyageai
from bson.binary import Binary, BinaryVectorDtype

# Initialize the VoyageAI Client
os.environ["VOYAGE_API_KEY"] = "<VOYAGEAI-API-KEY>"
vo = voyageai.Client()

# Define a function to generate embeddings for all strings in `texts`
def generate_embeddings(texts, model: str, dtype: str, output_dimension: int):
    embeddings = []
    for text in texts:  # Process eachstring in the data list
        embedding = vo.embed(
            texts=[text],  # Pass each string as a list with a single item
            model=model,
            output_dtype=dtype,
            output_dimension=output_dimension,
        ).embeddings[0]
        embeddings.append(embedding)  # Collect the embedding for the current text
    return embeddings

# Convert embeddings to BSON vectors
def generate_bson_vector(vector, vector_dtype):
   return Binary.from_vector(vector, vector_dtype)

In [ ]:
# Load sample data
data = [
   "The Great Wall of China is visible from space.",
   "The Eiffel Tower was completed in Paris in 1889.",
   "Mount Everest is the highest peak on Earth at 8,848m.",
   "Shakespeare wrote 37 plays and 154 sonnets during his lifetime.",
   "The Mona Lisa was painted by Leonardo da Vinci."
]

In [ ]:
# Use the function with different output data types to generate embeddings
model_name = "voyage-4-large"
output_dimension = 1024

# Generate embeddings in all supported data types
float32_embeddings = generate_embeddings(data, model=model_name, dtype="float", output_dimension=output_dimension)
int8_embeddings = generate_embeddings(data, model=model_name, dtype="int8", output_dimension=output_dimension)
int1_embeddings = generate_embeddings(data, model=model_name, dtype="ubinary", output_dimension=output_dimension)

In [ ]:
# For all vectors in your collection, generate BSON vectors of float32, int8, and int1 embeddings
bson_float32_embeddings = []
bson_int8_embeddings = []
bson_int1_embeddings = []
for i, (f32_emb, int8_emb, int1_emb) in enumerate(zip(float32_embeddings, int8_embeddings, int1_embeddings)):
   bson_float32_embeddings.append(generate_bson_vector(f32_emb, BinaryVectorDtype.FLOAT32))
   bson_int8_embeddings.append(generate_bson_vector(int8_emb, BinaryVectorDtype.INT8))
   bson_int1_embeddings.append(generate_bson_vector(int1_emb, BinaryVectorDtype.PACKED_BIT))

In [ ]:
# Specify the field names for the float32, int8, and int1 embeddings
float32_field = "float32_embedding" 
int8_field = "int8_embedding"
int1_field = "int1_embedding"

# Define function to create documents with BSON vector embeddings
def create_new_docs_with_bson_vectors(bson_float32_embeddings, bson_int8_embeddings, bson_int1_embeddings, data):
  docs = []
  for i, (bson_f32_emb, bson_int8_emb, bson_int1_emb, text) in enumerate(zip(bson_float32_embeddings, bson_int8_embeddings, bson_int1_embeddings, data)):

     doc = {
          "_id": i,
          "text": text,
          float32_field: bson_f32_emb,
          int8_field: bson_int8_emb,
          int1_field: bson_int1_emb
     }
     docs.append(doc)
  return docs

# Create the documents
documents = create_new_docs_with_bson_vectors(bson_float32_embeddings, bson_int8_embeddings, bson_int1_embeddings, data)

In [ ]:
import pymongo

mongo_client = pymongo.MongoClient("<CONNECTION-STRING>")
# Insert documents into a new database and collection
db = mongo_client["<DATABASE-NAME>"]
collection_name = "<COLLECTION-NAME>"
db.create_collection(collection_name)
collection = db[collection_name]

collection.insert_many(documents)

In [ ]:
from pymongo.operations import SearchIndexModel
import time

# Define and create the vector search index
index_name = "vector_index"
search_index_model = SearchIndexModel(
  definition={
    "fields": [
      {
        "type": "vector",
        "path": float32_field,
        "similarity": "dotProduct",
        "numDimensions": 1024
      },
      {
        "type": "vector",
        "path": int8_field,
        "similarity": "dotProduct",
        "numDimensions": 1024
      },
      {
        "type": "vector",
        "path": int1_field,
        "similarity": "euclidean",
        "numDimensions": 1024
      }
    ]
  },
  name=index_name,
  type="vectorSearch"
)
result = collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")

# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None
if predicate is None:
  predicate = lambda index: index.get("queryable") is True
while True:
  indices = list(collection.list_search_indexes(index_name))
  if len(indices) and predicate(indices[0]):
    break
  time.sleep(5)
print(result + " is ready for querying.")

In [ ]:
import voyageai
from bson.binary import Binary, BinaryVectorDtype

# Define a function to run a vector search query
def run_vector_search(query_text, collection, path):
    # Map path to output dtype and BSON vector type
    path_to_dtype = {
        float32_field: ("float", BinaryVectorDtype.FLOAT32),
        int8_field: ("int8", BinaryVectorDtype.INT8),
        int1_field: ("ubinary", BinaryVectorDtype.PACKED_BIT),
    }

    if path not in path_to_dtype:
        raise ValueError("Invalid path. Must be one of float32_field, int8_field, int1_field.")

    # Get Voyage AI output dtype and BSON vector type based on the path
    output_dtype, bson_dtype = path_to_dtype[path]

    # Generate query embeddings using Voyage AI
    query_vector = vo.embed(
        texts=[query_text],
        model="voyage-4-large",
        input_type="query",
        output_dtype=output_dtype
    ).embeddings[0]

    # Convert the query vector to BSON format
    bson_query_vector = Binary.from_vector(query_vector, bson_dtype)

    # Define the aggregation pipeline for vector search
    pipeline = [
        {
            "$vectorSearch": {
                "index": index_name,  # Replace with your index name
                "path": path,         # Path to the embedding field
                "queryVector": bson_query_vector,  # BSON-encoded query vector
                "numCandidates": 5,
                "limit": 2
            }
        },
        {
            "$project": {
                "_id": 0,
                "text": 1,
                "score": { "$meta": "vectorSearchScore" }  # Include the similarity score
            }
        }
    ]

    # Run the aggregation pipeline and return results
    return collection.aggregate(pipeline)

In [ ]:
from pprint import pprint

# Define a list of embedding fields to query
embedding_fields = [float32_field, int8_field, int1_field] 
results = {}

# Run vector search queries for each embedding type
query_text = "science fact"
for field in embedding_fields:
    results[field] = list(run_vector_search(query_text, collection, field)) 

# Print the results
for field, field_results in results.items():
    print(f"Results from {field}")
    pprint(field_results)